In [1]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

from qiskit.synthesis import SuzukiTrotter
from qiskit.quantum_info import Statevector
from qiskit.circuit import QuantumCircuit
from qiskit.circuit.library import PauliEvolutionGate
from qiskit import transpile

from graph_cutter import *
from utils import *

from azure.quantum import Workspace
from azure.quantum.target.microsoft import MicrosoftEstimator

workspace = Workspace (
   resource_id = "/subscriptions/e0d51919-2dda-4b7e-a8e7-77cc18580acb/resourceGroups/AzureQuantum/providers/Microsoft.Quantum/Workspaces/QECBenchmarking",
   location = "uksouth"
)

estimator = MicrosoftEstimator(workspace)

params = estimator.make_params(num_items=1)

second_order_formula = SuzukiTrotter()
L = 3
H = get_hamiltonian(L=L, J=0.2, h=1.0, alpha=np.pi / 8)
final_time = np.pi
num_timesteps = 10
dt = final_time / num_timesteps
initial_state = Statevector.from_label("000")

trotter_step_second_order = PauliEvolutionGate(H, dt, synthesis=second_order_formula)
circuit = QuantumCircuit(H.num_qubits)
for i in range(1):
    circuit.append(trotter_step_second_order, range(H.num_qubits))
target_basis = ['rx', 'ry', 'rz', 'h', 'cx']
circuit = transpile(circuit,
                       basis_gates=target_basis, 
                       optimization_level=1) 
# plot_trotter_info(circuit)
# circuit.draw('mpl', fold=-1)

In [2]:
# Error budget
params.error_budget = 0.01
params.constraints.max_t_factories = 1
job = estimator.submit(circuit, input_params=params)
r = job.get_results()
Q = r['physicalCounts']['physicalQubits']
T = r['physicalCounts']['runtime']*1e-3
print('\n', 'total physical qubits required: ', Q, '\n', 'total runtime', T)

......
 total physical qubits required:  6824 
 total runtime 5680.400000000001


In [3]:
# seeds = range(100,111)
T_factory = range(1,6)
final_Q = Q
final_t = T
overall_T_fact = [0]*len(T_factory)
best_budget = 0.01
# for sed in seeds:
for _ in range(10):
    subcircuits, g, w = find_cuts(circuit, max_cut=8, plot=False)
    subcircuits = check_estimation_validity(subcircuits)
    print('partition: ',[len(s.data) for s in subcircuits])
    sub_error_budget = error_budget_partition(params.error_budget, len(subcircuits))
    sub_params = estimator.make_params(num_items=len(T_factory))
    print('Sampling Overhead: ', 16**w*9**g)
    best_T_fact = [0]*len(T_factory)
    for edx, e in enumerate(sub_error_budget):
        q = [Q]*len(subcircuits)
        t = [T]*len(subcircuits)
        T_list = [0]*len(subcircuits)
        # print('\n', 'with error budget partition: ', e)
        # print('\n', 'Estimating')
        for sdx, s in enumerate(subcircuits):
            for tdx, tf in enumerate(T_factory):
                sub_params.items[tdx].error_budget = e[sdx]*0.001
                sub_params.items[tdx].constraints.max_t_factories = tf
            job = estimator.submit(s, input_params=sub_params)
            
            r = job.get_results()
            print(s.data)
            q[sdx] = min([r[tdx]['physicalCounts']['physicalQubits'] for tdx in range(len(T_factory))])
            for _index, value in enumerate([r[tdx]['physicalCounts']['physicalQubits'] for tdx in range(len(T_factory))]):
                if  q[sdx] == value:
                    T_list[sdx] = T_factory[_index]
                    t[sdx] = r[_index]['physicalCounts']['runtime']*1e-3

        # print('\n', 'total physical qubits required: ', max(q), '\n', r'total runtime ($\mu$ s): ', sum(t), '\n', 'with T factory:', T_list)
        if max(q) < Q:
            # best_seed = sed
            best_T_fact = T_list
            Q = max(q)
            T = sum(t)
    print('lowest combination: ', 'Max T factories--', best_T_fact, 'total physical qubits--', Q, r'total runtime ($\mu$ s): ', T)
    print('----------------------------------------------------------------')
    print('\n')
    if final_Q > Q:
            # best_seed = sed
            best_budget = e
            overall_T_fact = best_T_fact
            final_Q = Q
            final_t = T
print('\n')
print('----------------------------------------------------------------')
print('lowest combination: error budget--', best_budget, 'Max T factories--', overall_T_fact, 'total physical qubits--', Q, r'total runtime ($\mu$ s): ', T)


partition:  [4, 1, 15]
Sampling Overhead:  9437184
......[CircuitInstruction(operation=Instruction(name='rz', num_qubits=1, num_clbits=0, params=[-0.06283185307179587]), qubits=(Qubit(QuantumRegister(3, 'q'), 2),), clbits=()), CircuitInstruction(operation=Instruction(name='cx', num_qubits=2, num_clbits=0, params=[]), qubits=(Qubit(QuantumRegister(3, 'q'), 1), Qubit(QuantumRegister(3, 'q'), 2)), clbits=()), CircuitInstruction(operation=Instruction(name='rz', num_qubits=1, num_clbits=0, params=[-1.6910198727717658]), qubits=(Qubit(QuantumRegister(3, 'q'), 1),), clbits=()), CircuitInstruction(operation=Instruction(name='ry', num_qubits=1, num_clbits=0, params=[0.5804906304278862]), qubits=(Qubit(QuantumRegister(3, 'q'), 1),), clbits=())]
......[CircuitInstruction(operation=Instruction(name='rz', num_qubits=1, num_clbits=0, params=[-0.06283185307179587]), qubits=(Qubit(QuantumRegister(2, 'q'), 1),), clbits=())]
.......[CircuitInstruction(operation=Instruction(name='cx', num_qubits=2, num_c

NameError: name 'best_budget' is not defined

In [ ]:
# from qiskit.circuit import Qubit, CircuitInstruction, Instruction
# qc = QuantumCircuit(2)
# qc.data.insert(0,(Instruction(name='cx', num_qubits=2, num_clbits=0, params=[]), [Qubit(QuantumRegister(2, 'q'), 0), Qubit(QuantumRegister(2, 'q'), 1)], []))
# test = [qc]
# check_estimation_validity(test)

[]